In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
to_generate_data = False
to_train = False
to_test = True

*Data Preparation*

In [5]:
if to_generate_data:
    df = pd.read_parquet('autodl-tmp/mins_data.parquet')
    df.sort_values(by=['START_TIME', 'Attraction'], inplace=True)
    df['lag1'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(1).fillna(0)
    df['lag2'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(2).fillna(0)
    df.reset_index(inplace = True,drop = True)
    one_hot_columns = [col for col in df.columns if 
                       pd.api.types.is_numeric_dtype(df[col]) and 
                       sorted(df[col].unique()) == [0, 1]]
    non_one_hot_columns = [col for col in df.columns if 
                           pd.api.types.is_numeric_dtype(df[col]) and 
                           col not in one_hot_columns and col not in ['lag1','lag2','WAIT_TIME_MAX']]
    scaler = StandardScaler()
    df[non_one_hot_columns] = scaler.fit_transform(df[non_one_hot_columns])
    
    num_attractions = len(df.Attraction.unique())
    grouped = df.groupby('START_TIME')
    feature_columns = [i for i in list(df) if i not in ['WORK_DATE',
     'START_TIME',
     'Attraction',
     'WAIT_TIME_MAX',
     'DEB_TIME',
     'FIN_TIME',
     'DEB_TIME_x',
     'FIN_TIME_x',
     'DEB_TIME_y',
     'FIN_TIME_y']]
    zero_data = np.zeros((1, len(feature_columns)))
    zero_df = pd.DataFrame(zero_data, columns=feature_columns)
    processed_groups = []
    targets = []
    all_attractions = sorted(df['Attraction'].unique())
    for time_point, group in tqdm(grouped):
        processed_group = group.copy()
        processed_group = processed_group.drop_duplicates(subset=['Attraction'], keep='first')
        present_attractions = group['Attraction'].unique()
        missing_attractions = set(all_attractions) - set(present_attractions)
        for attraction in missing_attractions:
            temp_df = zero_df.copy()
            temp_df['Attraction'] = attraction
            temp_df['START_TIME'] = time_point
            processed_group = pd.concat([processed_group, temp_df], ignore_index=True)
        processed_group.sort_values(by=['Attraction'], inplace=True)
        t = processed_group.pop('WAIT_TIME_MAX').fillna(0)
        data = torch.tensor(processed_group[feature_columns].values, dtype=torch.float)
        processed_groups.append(data)
        targets.append(t)
        if data.shape[0] != 26:
            break
    features_tensor = torch.stack(processed_groups)
    labels_tensor = torch.tensor(np.array(targets))
    total_samples = len(features_tensor)
    # last 10% for validation
    split_idx = int(total_samples * 0.9)
    
    train_features = features_tensor[:split_idx]
    train_labels = labels_tensor[:split_idx]
    validation_features = features_tensor[split_idx:]
    validation_labels = labels_tensor[split_idx:]
    
    train_dataset = TensorDataset(train_features, train_labels)
    validation_dataset = TensorDataset(validation_features, validation_labels)
    torch.save(train_dataset, 'autodl-tmp/train_dataset.pth')
    torch.save(validation_dataset, 'autodl-tmp/validation_dataset.pth')
else:
    df = pd.read_parquet('autodl-tmp/mins_data.parquet')
    df.sort_values(by=['START_TIME', 'Attraction'], inplace=True)
    df['lag1'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(1).fillna(0)
    df['lag2'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(2).fillna(0)
    df.reset_index(inplace = True,drop = True)
    one_hot_columns = [col for col in df.columns if 
                       pd.api.types.is_numeric_dtype(df[col]) and 
                       sorted(df[col].unique()) == [0, 1]]
    non_one_hot_columns = [col for col in df.columns if 
                           pd.api.types.is_numeric_dtype(df[col]) and 
                           col not in one_hot_columns and col not in ['lag1','lag2','WAIT_TIME_MAX']]
    scaler = StandardScaler()
    df[non_one_hot_columns] = scaler.fit_transform(df[non_one_hot_columns])
    
    num_attractions = len(df.Attraction.unique())
    grouped = df.groupby('START_TIME')
    feature_columns = [i for i in list(df) if i not in ['WORK_DATE',
     'START_TIME',
     'Attraction',
     'WAIT_TIME_MAX',
     'DEB_TIME',
     'FIN_TIME',
     'DEB_TIME_x',
     'FIN_TIME_x',
     'DEB_TIME_y',
     'FIN_TIME_y']]
    train_dataset = torch.load('autodl-tmp/train_dataset.pth')
    validation_dataset = torch.load('autodl-tmp/validation_dataset.pth')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

*Network design*

In [6]:
class Network1(nn.Module):
    def __init__(self, input_size, output_size, num_layers=1):
        super(Network1, self).__init__()
        self.fc1 = nn.Linear(input_size, 1500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1500, 1000)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1000, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
    def forward(self, x, return_intermediate=False):
        intermediate_outputs = {}
        out = self.relu(self.fc1(x))
        intermediate_outputs['fc1'] = out
        out = self.dropout1(out)
        intermediate_outputs['dropout1'] = out
        out = self.relu(self.fc2(out))
        intermediate_outputs['fc2'] = out
        out = self.dropout2(out)
        intermediate_outputs['dropout2'] = out
        out = self.fc3(out)
        intermediate_outputs['fc3'] = out
        
        if return_intermediate:
            return intermediate_outputs
        else:
            return out

class Network2(nn.Module):
    def __init__(self,input_size,output_size):
        super(Network2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256,128)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(128, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='leaky_relu')
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout1(out)
        out = self.relu(self.fc2(out))
        out = self.dropout2(out)
        out = self.relu(self.fc3(out))
        out = self.dropout3(out)
        out = self.fc4(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network1 = Network1(input_size=num_attractions*len(feature_columns),
                    output_size=num_attractions*num_attractions).to(device)
network2 = Network2(input_size=num_attractions, output_size=num_attractions).to(device)


In [7]:
def build_initial_matrix(out):
    matrix = out.view(26, 26)
    normalized_matrix = matrix / matrix.sum(dim=1, keepdim=True)
    return normalized_matrix
def calculate_stationary_matrix(matrix, num_iterations=30, epsilon=1e-6):
    v = torch.ones(matrix.size(0), dtype=matrix.dtype, device=matrix.device)
    v = v / v.sum()
    for _ in range(num_iterations):
        v_next = torch.mv(matrix, v)
        v_next = v_next / v_next.sum()
        if torch.norm(v - v_next) < epsilon:
            break
        v = v_next
    return v
def Middle(out):
    return calculate_stationary_matrix(build_initial_matrix(out))

In [8]:
criterion = nn.MSELoss()
optimizer1 = optim.Adam(network1.parameters(), lr=0.001)
optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

*Evaluation Design*

In [14]:
def validate(model1, model2, validation_loader, criterion, device):
    model1.eval() 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in validation_loader:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = model1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = model2(stationary_matrix)
            loss = criterion(predictions, targets)
            val_loss += loss.item()
    return np.sqrt(val_loss / len(validation_loader))
def RMSE_for_Attraction(network1,network2):
    attraction = df.Attraction.unique()
    attraction.sort()
    network1.eval() 
    network2.eval()
    result = pd.DataFrame({'attraction':attraction,'RMSE':[0 for i in attraction]})
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in tqdm(validation_loader):
            targets = targets.reshape(26)
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = network1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = network2(stationary_matrix)
            for i in range(len(predictions)):
                index = result['attraction'] == attraction[i]
                result.loc[index, 'RMSE'] += float((targets[i] - predictions[i]))**2
    result.RMSE = result.RMSE/len(validation_loader)
    result.RMSE = result.RMSE.apply(lambda x:np.sqrt(x))
    return result

*Model Training*

In [15]:
if to_train:
    num_epochs = 10
    current_best = 1e10
    RMSE_val = []
    for epoch in range(num_epochs):
        network1.train()
        network2.train()
        train_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in pbar:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            out1 = network1(inputs)
            stationary_matrix = Middle(out1)
            predictions = network2(stationary_matrix).reshape((1,26)).float()
            loss = criterion(predictions, targets)
            loss.backward()
            clip_value = 1 #1-10
            nn.utils.clip_grad_norm_(network1.parameters(), clip_value) 
            nn.utils.clip_grad_norm_(network2.parameters(), clip_value)
            optimizer1.step()
            optimizer2.step()
            train_loss += loss.item()
            pbar.set_postfix({'Train Loss': train_loss / (batch_idx + 1)})
            
        val_loss = validate(network1, network2, validation_loader, criterion, device)
        RMSE_val.append(val_loss)
        if np.sqrt(val_loss) < current_best:
            torch.save(network1, 'network1_v1.pth')
            torch.save(network2, 'network2_v1.pth')
            current_best = val_loss
        print(f"Validation Loss: {val_loss:.4f}")

*Best Model Evaluation*

In [16]:
if to_test:
    best_network1 = torch.load('network1_v1.pth')
    best_network2 = torch.load('network2_v1.pth')
    result = RMSE_for_Attraction(best_network1,best_network2)
    print('RMSE for all Attractions', result.RMSE.mean())
    print(result)

100%|██████████| 4060/4060 [01:17<00:00, 52.26it/s]

RMSE for all Attractions 24.495744782447243
          attraction       RMSE
0        Bumper Cars  12.496342
1        Bungee Jump  28.619547
2       Circus Train   6.192073
3        Crazy Dance   0.044842
4      Dizzy Dropper  16.778949
5         Drop Tower  35.348845
6     Flying Coaster  20.745494
7          Free Fall  49.920049
8        Giant Wheel  49.385965
9       Giga Coaster   0.070613
10          Go-Karts  30.489797
11     Haunted House  22.301925
12     Himalaya Ride   0.018071
13  Inverted Coaster  23.829943
14    Kiddie Coaster  13.275166
15    Merry Go Round  18.537053
16        Oz Theatre   6.540795
17       Rapids Ride  22.499213
18    Roller Coaster  26.260995
19  Spinning Coaster  24.238649
20      Spiral Slide  88.738792
21     Superman Ride  10.953985
22        Swing Ride  50.071282
23     Vertical Drop   8.849341
24        Water Ride  27.440018
25           Zipline  43.241618


In [19]:
result.to_csv('version1result.csv')